In [ ]:
import torch
import pyg_lib

print("PyTorch Version:", torch.__version__)
print("pyg-lib Version:", pyg_lib.__version__)


In [ ]:
# # # Install required packages.
# !pip install torch==2.6.0
# !pip install torch-geometric torch-sparse torch-scatter torch-cluster torch-spline-conv pyg-lib -f https://data.pyg.org/whl/torch-2.4.0+cpu.html
# !pip install pytorch_frame
# !pip install -U sentence-transformers # we need another package for text encoding


In [ ]:
import os
import math
import numpy as np
from tqdm import tqdm

import torch
import torch_geometric
import torch_frame


In [ ]:

from torch.nn import BCEWithLogitsLoss, L1Loss
from relbench.datasets import get_dataset
from relbench.tasks import get_task

dataset = get_dataset("rel-f1", download=True)
task = get_task("rel-f1", "driver-position", download=True)

train_table = task.get_table("train")
val_table = task.get_table("val")
test_table = task.get_table("test")

out_channels = 1
loss_fn = L1Loss()
tune_metric = "rmse"
higher_is_better = False

Let's check out the training table just to make sure it looks fine.

In [ ]:
train_table

Note that to load the data we did not require any deep learning libraries. Now we introduce the PyTorch Frame library, which is useful for encoding individual tables into initial node features.

In [ ]:
# Some book keeping
from torch_geometric.seed import seed_everything

seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # check that it's cuda if you want it to run in reasonable time!
root_dir = ".tutorials/data"

The first big move is to build a graph out of the database. Here we use our pre-prepared conversion function.

The source code can be found at: https://github.com/snap-stanford/relbench/blob/main/relbench/modeling/graph.py

Each node in the graph corresonds to a single row in the database. Crucially, PyTorch Frame stores whole tables as objects in a way that is compatibile with PyG minibatch sampling, meaning we can sample subgraphs as in https://arxiv.org/abs/1706.02216, and retrieve the relevant raw features.

PyTorch Frame also stores the `stype` (i.e., modality) of each column, and any specialized feature encoders (e.g., text encoders) to be used later. So we need to configure the `stype` for each column, for which we use a function that tries to automatically detect the `stype`.

In [ ]:
from relbench.modeling.utils import get_stype_proposal

db = dataset.get_db()
col_to_stype_dict = get_stype_proposal(db)
col_to_stype_dict

If trying a new dataset, you should definitely check through this dict of `stype`s to check that look right, and manually change any mistakes by the auto-detection function.

Next we also define our text encoding model, which we use GloVe embeddings for speed and convenience. Feel free to try alternatives here.

In [ ]:
from typing import List, Optional
from sentence_transformers import SentenceTransformer
from torch import Tensor


class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                            ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device if torch.cuda.is_available() else "cpu"
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

In [ ]:
import os
import pickle

output_file = 'output_fi.pkl'
cached = True

if os.path.exists(output_file) and cached:
    # File exists: load the data
    with open(output_file, 'rb') as f:
        data, col_stats_dict = pickle.load(f)
    print("Loaded data from file.")
else:
    # File does not exist: run the code and save the output
    from torch_frame.config.text_embedder import TextEmbedderConfig
    from relbench.modeling.graph import make_snapshot_graph
    # Ensure GloveTextEmbedding, device, db, col_to_stype_dict, and root_dir are defined
    text_embedder_cfg = TextEmbedderConfig(
        text_embedder=GloveTextEmbedding(device=device), batch_size=256
    )
    
    data, col_stats_dict = make_snapshot_graph(
        db,
        col_to_stype_dict=col_to_stype_dict,              # specified column types
        main_table_name="races",                          # use 'races' table as timestamp reference
        interval_days=720,                                 # generate snapshots every 30 days
        text_embedder_cfg=text_embedder_cfg,              # chosen text encoder
        cache_dir=os.path.join(root_dir, "rel-f1_materialized_cache"),  # store materialized graph for convenience
    )
    
    # Save the output to a file for future use
    with open(output_file, 'wb') as f:
        pickle.dump((data, col_stats_dict), f)
    print("Data computed and saved to file.")


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

def visualize_hetero_graph(snapshot, num_nodes=10):
    """Visualize a small portion of a heterogeneous graph with a specific edge type."""
    
    # Select the first available edge type (e.g., ('node_type1', 'relation', 'node_type2'))
    edge_type = list(snapshot.edge_index_dict.keys())[0]
    print(f"Using edge type: {edge_type}")  # Debugging

    # Extract edge index for the selected edge type
    edge_index = snapshot[edge_type].edge_index
    
    # Convert the selected subgraph to NetworkX
    nx_graph = nx.Graph()  # Create an empty NetworkX graph

    # Add edges from edge_index
    for i in range(edge_index.shape[1]):
        src = int(edge_index[0, i].item())
        dst = int(edge_index[1, i].item())
        if edge_index.numel() == 0:
            print(f"⚠️ Warning: Edge type {edge_type} has no edges! Skipping visualization.")
            return
        nx_graph.add_edge(src, dst)  # Add edge to the NetworkX graph

    # Get a subset of nodes
    sampled_nodes = list(nx_graph.nodes)[:num_nodes]
    subgraph = nx_graph.subgraph(sampled_nodes)
    
    # Plot the graph
    plt.figure(figsize=(8, 6))
    nx.draw(subgraph, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=10)
    plt.title(f"Visualization of {num_nodes} nodes from edge type {edge_type}")
    plt.show()

# Select the first snapshot
snapshot = data[1]  # Assuming data is a list of snapshots

# Call the function
visualize_hetero_graph(snapshot, num_nodes=150)


We can now check out `data`, our main graph object. `data` is a heterogeneous and temporal graph, with node types given by the table it originates from.

In [ ]:
data[1]

We can also check out the TensorFrame for one table like this:

In [ ]:
col_stats_dict

This may be a little confusing at first, as in graph ML it is more standard to associate to the graph object `data` a tensor, e.g., `data.x` for which `data.x[idx]` is a 1D array/tensor storing all the features for node with index `idx`.

But actually this `data` object behaves similarly. For a given node type, e.g., `races` again, `data['races']` stores two pieces of information


In [ ]:
print(len(data))  # Number of snapshots created


A `TensorFrame` object, and a timestamp for each node. The `TensorFrame` object acts analogously to the usual tensor of node features, and you can simply use indexing to retrieve the features of a single row (node), or group of nodes.

In [ ]:
data[1]["races"].tf[10:20]

We can also check the edge indices between two different node types, such as `races` amd `circuits`. Note that the edges are also heterogenous, so we also need to specify which edge type we want to look at. Here we look at `f2p_curcuitId`, which are the directed edges pointing _from_ a race (the `f` stands for `foreign key`), _to_ the circuit at which te race happened (the `p` stands for `primary key`).

In [ ]:
data.__sizeof__()

In [ ]:
for edge_type in data[0].edge_types:
    print(f"Edge: {edge_type}, Shape: {data[0][edge_type].edge_index.shape}")


In [ ]:
for node_type in data[14].node_types:
    print(f"Node: {node_type}, Feature Shape: {data[14][node_type].tf}")


In [ ]:
data[1][("races", "f2p_circuitId", "circuits")]

Now we are ready to instantiate our data loaders. For this we will need to import PyTorch Geometric, our GNN library. Whilst we're at it let's add a seed.


In [ ]:
from relbench.modeling.graph import get_node_train_table_input
from torch_geometric.loader import NeighborLoader
import torch

loader_dict = {}
for split, table in [("train", train_table), ("val", val_table), ("test", test_table)]:
    table_input = get_node_train_table_input(table=table, task=task)
    entity_table = table_input.nodes[0]  # 'drivers'
    loader_dict[split] = []

    # Get driverIds from the table
    driver_ids = torch.tensor(table.df['driverId'].values, dtype=torch.long)

    for snapshot_idx, snapshot in enumerate(data):
        time_attr = "time" if table_input.time is not None and "time" in snapshot.get(entity_table, {}) else None
        if time_attr is None and 'dob' in snapshot.get(entity_table, {}):
            time_attr = 'dob'

        if snapshot[entity_table].num_nodes is None or snapshot[entity_table].num_nodes == 0:
            print(f"⚠️ No nodes for {entity_table}. Skipping snapshot {snapshot_idx}.")
            continue
        print(f"Available node types in snapshot {snapshot_idx}: {snapshot.node_types}")
        if entity_table not in snapshot.node_types:
            print(f"⚠️ Warning: {entity_table} not found in snapshot {snapshot_idx}. Skipping.")
            continue

        # Map driverIds to snapshot node indices (assuming 'n_id' or similar exists)
        if 'n_id' not in snapshot[entity_table]:
            print(f"⚠️ Warning: 'n_id' not found in {entity_table} for snapshot {snapshot_idx}. Assuming sequential IDs.")
            snapshot[entity_table].n_id = torch.arange(snapshot[entity_table].num_nodes)
        
        # Find indices of driverIds in the snapshot
        snapshot_driver_ids = snapshot[entity_table].n_id
        mask = torch.isin(snapshot_driver_ids, driver_ids)
        input_node_indices = snapshot_driver_ids[mask]

        if len(input_node_indices) == 0:
            print(f"⚠️ No matching {entity_table} nodes in snapshot {snapshot_idx} for {split} table.")
            continue

        loader = NeighborLoader(
            snapshot,
            num_neighbors=[10, 10],
            time_attr=time_attr,
            input_nodes=(entity_table, input_node_indices),  # Restrict to matching nodes
            input_time=table_input.time if time_attr is not None else None,
            transform=table_input.transform,
            batch_size=512,
            temporal_strategy="uniform",
            shuffle=(split == "train"),
            num_workers=0,
            persistent_workers=False,
        )
        if not any(loader):
            print(f"⚠️ Loader for {entity_table} in snapshot {snapshot_idx} is empty!")
        else:
            print(f"Loader for snapshot {snapshot_idx} created with {len(input_node_indices)} input nodes.")
        loader_dict[split].append(loader)

# Debug loaders
for split in ["train", "val", "test"]:
    print(f"\nChecking {split} loaders:")
    for i, snapshot_loader in enumerate(loader_dict[split]):
        print(f"  Snapshot {i}:")
        batches = list(snapshot_loader)
        print(f"    Number of batches: {len(batches)}")
        for j, batch in enumerate(batches[:2]):  # Limit to first 2 batches
            print(f"    Batch {j}: {batch}")

In [ ]:
print(f"Available node types in snapshot[0]: {data[0].node_types}")


Now we need our model...




In [ ]:
from typing import Any, Dict, List
import torch
from torch import nn, Tensor
from torch.nn import L1Loss, Embedding, ModuleDict
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_frame.data.stats import StatType
from torch_geometric.typing import NodeType, EdgeType
from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, LSTMBasedTemporalEncoder
from torch_frame.nn import TabTransformer
from torch.nn import LayerNorm, Sequential, Linear, ReLU
from torch_frame import stype  # must be imported

#############################################
# Custom TabTransformer
#############################################
class CustomTabTransformer(TabTransformer):
    def __init__(self, *args, num_encoder_out_dim=None, **kwargs):
        self.channels = kwargs.get("channels")
        if self.channels is None:
            raise ValueError("channels must be provided to CustomTabTransformer")
        super().__init__(*args, **kwargs)
        if hasattr(self, 'num_encoder') and self.num_encoder is not None:
            if num_encoder_out_dim is None:
                num_encoder_out_dim = 1
            self.num_proj = Linear(num_encoder_out_dim, self.channels)
        else:
            self.num_proj = None

        # Compute the total input size from all encoders.
        input_size = 0
        if hasattr(self, 'cat_encoder') and self.cat_encoder is not None:
            input_size += self.channels
        if hasattr(self, 'num_encoder') and self.num_encoder is not None:
            input_size += self.channels
        if hasattr(self, 'multi_cat_encoder') and self.multi_cat_encoder is not None:
            input_size += self.channels
        if hasattr(self, 'text_embedded_encoder') and self.text_embedded_encoder is not None:
            input_size += self.channels
        if hasattr(self, 'timestamp_encoder') and self.timestamp_encoder is not None:
            input_size += self.channels

        out_channels = kwargs["out_channels"]
        self.decoder = Sequential(
            Linear(input_size, out_channels),
            ReLU(),
            LayerNorm(out_channels),
            Linear(out_channels, out_channels),
        )
        print(f"Decoder input size: {input_size}, out_channels: {out_channels}")

    def forward(self, tf):
        xs = []
        # Process categorical features.
        if hasattr(self, 'cat_encoder') and self.cat_encoder is not None:
            if hasattr(tf, "feat_dict") and stype.categorical in tf.feat_dict:
                cat_data = tf.feat_dict[stype.categorical]
            else:
                raise ValueError("No categorical features found in the TensorFrame using stype.categorical.")
            x_cat = self.cat_encoder(cat_data)
            if x_cat.dim() == 3:
                x_cat = x_cat.mean(dim=1)
            xs.append(x_cat)
        # Process numerical features.
        if hasattr(self, 'num_encoder') and self.num_encoder is not None:
            if hasattr(tf, "feat_dict") and stype.numerical in tf.feat_dict:
                num_data = tf.feat_dict[stype.numerical]
            else:
                raise ValueError("No numerical features found in the TensorFrame using stype.numerical.")
            x_num = self.num_encoder(num_data)
            if self.num_proj is not None:
                x_num = self.num_proj(x_num)
            if x_num.dim() == 3:
                x_num = x_num.mean(dim=1)
            xs.append(x_num)
        if xs:
            x = torch.cat(xs, dim=1)
        else:
            raise ValueError("No valid features were encoded!")
        out = self.decoder(x)
        return out

#############################################
# Model Definition
#############################################
class Model(nn.Module):
    def __init__(
        self,
        data: List[HeteroData],
        col_to_stype_dict: Dict[str, Dict[str, 'stype']],
        col_stats_dict: Dict[str, Dict[str, Dict[StatType, Any]]],
        num_layers: int,
        channels: int,
        out_channels: int,
        aggr: str,
        norm: str,
        shallow_list: List[NodeType] = [],
        id_awareness: bool = False,
    ):
        super().__init__()
        # Build mapping: node type -> {stype: [column names]}
        node_to_col_names_dict = {}
        for node_type, col_to_stype in col_to_stype_dict.items():
            col_names_dict = {}
            for col, stype_val in col_to_stype.items():
                if stype_val not in col_names_dict:
                    col_names_dict[stype_val] = []
                col_names_dict[stype_val].append(col)
            node_to_col_names_dict[node_type] = col_names_dict

        self.encoder = HeteroEncoder(
            channels=channels,
            node_to_col_names_dict=node_to_col_names_dict,
            node_to_col_stats=col_stats_dict,
            col_to_stype_dict=col_to_stype_dict,
            torch_frame_model_cls=CustomTabTransformer,
            torch_frame_model_kwargs={
                "num_layers": num_layers,
                "out_channels": channels,
                "num_heads": 4,
                "attn_dropout": 0.1,
                "ffn_dropout": 0.1,
                "num_encoder_out_dim": 1,
            },
        )

        self.temporal_encoder = LSTMBasedTemporalEncoder(
            node_types=[node for node in data[0].node_types if "time" in data[0][node]],
            channels=channels
        )

        self.gnn = HeteroGraphSAGE(
            node_types=data[0].node_types,
            edge_types=data[0].edge_types,
            channels=channels,
            aggr=aggr,
            num_layers=num_layers,
        )

        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1,
        )

        self.embedding_dict = ModuleDict({
            node: Embedding(data[0].num_nodes_dict.get(node, 0), channels)
            for node in shallow_list if data[0].num_nodes_dict.get(node, 0) > 0
        })

        self.id_awareness_emb = None
        if id_awareness:
            self.id_awareness_emb = nn.Embedding(1, channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.temporal_encoder.reset_parameters()
        self.gnn.reset_parameters()
        self.head.reset_parameters()
        for emb in self.embedding_dict.values():
            nn.init.normal_(emb.weight, std=0.1)
        if self.id_awareness_emb is not None:
            self.id_awareness_emb.reset_parameters()
                    
    def forward(self, batch: HeteroData, entity_table: NodeType) -> Tensor:
        seed_time = batch.time_dict.get(entity_table, None)
        # Encode features from each node type that has a TensorFrame.
        x_dict = self.encoder({
            node_type: batch[node_type].tf 
            for node_type in batch.node_types if hasattr(batch[node_type], 'tf')
        })
        # Add shallow embeddings.
        for node_type, emb in self.embedding_dict.items():
            if node_type not in x_dict or x_dict[node_type] is None:
                n = batch[node_type].num_nodes if hasattr(batch[node_type], 'num_nodes') else 0
                device = next(self.parameters()).device
                x_dict[node_type] = emb(torch.arange(n, device=device))
            else:
                n = batch[node_type].num_nodes if hasattr(batch[node_type], 'num_nodes') else x_dict[node_type].size(0)
                device = next(self.parameters()).device
                emb_tensor = emb(torch.arange(n, device=device))
                if x_dict[node_type].size(0) < n:
                    pad = torch.zeros((n - x_dict[node_type].size(0), x_dict[node_type].size(1)), device=device)
                    x_dict[node_type] = torch.cat([x_dict[node_type], pad], dim=0)
                x_dict[node_type] = x_dict[node_type] + emb_tensor

        # Ensure each node type has a feature tensor with expected number of rows.
        for node_type in batch.node_types:
            if not hasattr(batch[node_type], 'num_nodes'):
                continue
            expected = batch[node_type].num_nodes
            if node_type not in x_dict or x_dict[node_type] is None:
                device = next(self.parameters()).device
                x_dict[node_type] = torch.zeros((expected, 128), device=device)
            else:
                if x_dict[node_type].size(0) < expected:
                    pad = torch.zeros((expected - x_dict[node_type].size(0), x_dict[node_type].size(1)),
                                      device=x_dict[node_type].device)
                    x_dict[node_type] = torch.cat([x_dict[node_type], pad], dim=0)
        
        # Optionally apply temporal encoding.
        rel_time_dict = {}
        if seed_time is not None:
            rel_time_dict = self.temporal_encoder(x_dict, batch.time_dict, batch.batch_dict)
        for node_type, rel_time in rel_time_dict.items():
            if node_type in x_dict:
                x_dict[node_type] = x_dict[node_type] + rel_time
            else:
                x_dict[node_type] = rel_time

        # Filter out edge relations with empty indices.
        filtered_edge_index_dict = {key: value for key, value in batch.edge_index_dict.items() if value.numel() > 0}
        
        # IMPORTANT: Only keep node types expected by the GNN.
        expected_node_types = set(self.gnn.norms[0].keys())
        x_dict = {k: v for k, v in x_dict.items() if k in expected_node_types}

        # Pass features through the GNN.
        x_dict = self.gnn(x_dict, filtered_edge_index_dict)
        n = seed_time.size(0) if seed_time is not None else x_dict[entity_table].size(0)
        return self.head(x_dict[entity_table][:n])

#############################################
# Instantiate model, optimizer, loss, etc.
#############################################
# Ensure 'data', 'col_to_stype_dict', 'col_stats_dict', 'device', etc. are defined.
model = Model(
    data=data,
    col_to_stype_dict=col_to_stype_dict,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
    id_awareness=True,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
loss_fn = L1Loss()
epochs = 10


We also need standard train/test loops

In [ ]:
from tqdm import tqdm
import numpy as np
import copy

def train() -> float:
    model.train()
    loss_accum = count_accum = 0
    for snapshot_loader in tqdm(loader_dict["train"]):
        for batch in snapshot_loader:
            batch = batch.to(device)
            print(f"Batch node types: {batch.node_types}")
            # Robust check for entity_table
            if task.entity_table not in batch.node_types:
                print(f"Warning: {task.entity_table} not in batch.node_types. Checking batch keys: {list(batch.keys())}")
                if task.entity_table not in batch:
                    raise ValueError(f"Entity table {task.entity_table} missing in batch!")
            
            optimizer.zero_grad()
            
            pred = model(batch, task.entity_table)
            pred = pred.view(-1) if pred.size(1) == 1 else pred

            # Ensure y is present
            if 'y' not in batch[task.entity_table]:
                raise ValueError(f"'y' missing in {task.entity_table} for batch!")
            
            loss = loss_fn(pred.float(), batch[task.entity_table].y.float())
            loss.backward()
            optimizer.step()

            loss_accum += loss.detach().item() * pred.size(0)
            count_accum += pred.size(0)

    return loss_accum / count_accum if count_accum > 0 else float("inf")

@torch.no_grad()
def test(loader):
    model.eval()
    pred_list = []
    for snapshot_loader in loader:
        for batch in snapshot_loader:
            batch = batch.to(device)
            # print(f"Pred shape for {task.entity_table}: {pred.shape}")
            pred = model(batch, task.entity_table)
            pred = pred.view(-1) if pred.size(1) == 1 else pred
            pred_list.append(pred.detach().cpu())
    if not pred_list:
        raise ValueError("No predictions generated. Check loader or model forward pass.")
    return torch.cat(pred_list, dim=0).numpy()

Now we are ready to train!

In [28]:
print("=== Training Epoch ===")
for epoch in range(epochs):
    train_loss = train()  # your train() function
    print(f"Epoch {epoch+1} Training Loss: {train_loss:.4f}")
    
    # Evaluate on validation set.
    val_pred = test(loader_dict["val"])  # your test() function
    print(f"Epoch {epoch+1} Validation Prediction Shape: {val_pred.shape}")
    
    # Optionally, print a few sample predictions.
    print("Sample Predictions:", val_pred[:5])
    
    # Optionally, print a summary of one batch.
    for batch in loader_dict["val"]:
        print("Batch Summary:")
        for node in batch.data.node_types:
            print(f"  {node}: num_nodes = {batch.data[node].num_nodes}")
        print("Edge Index Summary:")
        for edge_type, edge_index in batch.data.edge_index_dict.items():
            print(f"  {edge_type}: shape = {edge_index.shape}")
        break  # print summary for first batch only

# Test the model.
test_pred = test(loader_dict["test"])


=== Training Epoch ===


 13%|█▎        | 4/31 [00:00<00:00, 36.49it/s]

Batch node types: ['results', 'standings', 'races', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor

 26%|██▌       | 8/31 [00:00<00:00, 28.71it/s]

Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['results

...

 97%|█████████▋| 30/31 [00:02<00:00,  6.19it/s]

Batch node types: ['qualifying', 'results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Batch node types: ['qualifying', 'results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']


100%|██████████| 31/31 [00:02<00:00, 10.40it/s]

Batch node types: ['qualifying', 'results', 'standings', 'races', 'constructor_results', 'constructor_standings', 'drivers', 'constructors', 'circuits']
Epoch 10 Training Loss: 5.3573


Epoch 10 Validation Prediction Shape: (1457,)
Sample Predictions: [13.305592 13.315838 15.026462 16.480076 16.481104]
Batch Summary:
  results: num_nodes = 20323
  standings: num_nodes = 28115
  races: num_nodes = 820
  drivers: num_nodes = 857
  constructors: num_nodes = 211
  circuits: num_nodes = 77
Edge Index Summary:
  ('results', 'f2p_raceId', 'races'): shape = torch.Size([2, 23])
  ('results', 'f2p_driverId', 'drivers'): shape = torch.Size([2, 20323])
  ('results', 'f2p_constructorId', 'constructors'): shape = torch.Size([2, 20323])
  ('standings', 'f2p_raceId', 'races'): shape = torch.Size([2, 23])
  ('standings', 'f2p_driverId', 'drivers'): shape = torch.Size([2, 28115])
  ('races', 'f2p_circuitId', 'circuits'): shape = torch.Size([2, 820])


In [29]:
print("Sample Predictions:", val_pred[:5])


Sample Predictions: [13.305592 13.315838 15.026462 16.480076 16.481104]


In [30]:
print("Sample Predictions:", torch.tensor(val_pred[:5]).cpu().detach().numpy())


Sample Predictions: [13.305592 13.315838 15.026462 16.480076 16.481104]
